# Proyecto 2
 
- *Dayana Valentina Gonzalez Vargas*
- *Juan Manuel Ramirez*

## Librerias

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Cargar los Datos

In [4]:
df=pd.read_csv('emotions_by_voice_registers.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,X,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0,1,1,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,1,2,2,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,2,3,3,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,4,5,5,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,5,6,6,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  909 non-null    int64  
 1   Unnamed: 0    909 non-null    int64  
 2   X             909 non-null    int64  
 3   meanfreq      909 non-null    float64
 4   sd            909 non-null    float64
 5   median        909 non-null    float64
 6   Q25           909 non-null    float64
 7   Q75           909 non-null    float64
 8   IQR           909 non-null    float64
 9   skew          909 non-null    float64
 10  kurt          909 non-null    float64
 11  sp.ent        909 non-null    float64
 12  sfm           909 non-null    float64
 13  mode          909 non-null    float64
 14  centroid      909 non-null    float64
 15  meanfun       909 non-null    float64
 16  minfun        909 non-null    float64
 17  maxfun        909 non-null    float64
 18  meandom       909 non-null    

### Eliminación de datos que no son necesarios o son de tipo indíce.
Eliminaremos los datos del DataSet que son tipo indíce.

In [6]:
df = df.drop(columns = [ 'Unnamed: 0.1', 'Unnamed: 0', 'X'])
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,2.863717,0.923566,0.307220,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,3.878650,0.918848,0.298859,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,4.589995,0.919519,0.313069,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,3.680995,0.921361,0.329295,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,5.031744,0.926238,0.337047,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad


**Dividimos en dos, el primer conjuntos Y es el conjunto con la etiqueta y el segundo es el conjunto X que va a ser el conjunto de las variables independientes sin la etiqueta**

In [7]:
y = df['label']
y = pd.get_dummies( y, drop_first = True)
X = df.drop(columns =['label'])

### Crear los conjuntod de Train, Validación y Test.
A partir de la función *train_test_split* vamos a realizarla dos veces, la primera para dividir en test y train. La segunda para ese train anterior dividirlo en val y train.

In [8]:
X_train1, X_test, y_train1, y_test = train_test_split( X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split( X_train1, y_train1, test_size=0.20, random_state=42)

## Preparación de los datos

A continuación, se preparan los datos convirtiendo las particiones previamente definidas en tensores para aprovechar la funcionalidad de backpropagation y juntando las características (x) y los resultados (y) en un objeto de tipo MyDataSet. Este objeto permitirá manejar con mayor facilidad los datos al proveer una estructura sencilla con la información más relevante para los procesos de entrenamiento.

In [9]:
class MyDataset():
 
  def __init__(self,x,y):
    y=y.values
    X=x.values
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [10]:
train_sec=MyDataset(X_train,y_train)
test_sec=MyDataset(X_test,y_test)
val_sec=MyDataset(X_val,y_val)

Tras la conversión al objeto MyDataSet se convierten en objetos tipo DataLoader para poder utilizar métodos de evaluación y optimización de la librería pytorch. Se imprime la primera entrada de los datos para entender su forma y comportamiento.

In [11]:
train_data=DataLoader(
    train_sec,
    batch_size=1,
    shuffle=False,
 )

test_data=DataLoader(
    test_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

val_data=DataLoader(
    val_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [12]:
for i, (data, labels) in enumerate(test_data):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

torch.Size([3, 20]) torch.Size([3, 2])
tensor([[0.2276, 0.0391, 0.2321, 0.2119, 0.2580, 0.0461, 2.3979, 9.7464, 0.8642,
         0.1852, 0.2336, 0.2276, 0.1917, 0.0584, 0.2759, 0.8516, 0.1484, 4.8438,
         4.6953, 0.1747],
        [0.2048, 0.0441, 0.2082, 0.1876, 0.2297, 0.0421, 1.9322, 6.3588, 0.8904,
         0.3525, 0.2066, 0.2048, 0.1781, 0.0343, 0.2462, 0.6719, 0.0781, 4.4219,
         4.3438, 0.1271],
        [0.1672, 0.0607, 0.1612, 0.1167, 0.2118, 0.0951, 1.0096, 3.4679, 0.9367,
         0.4687, 0.0938, 0.1672, 0.1152, 0.0208, 0.2222, 0.5907, 0.0859, 5.3203,
         5.2344, 0.1471]]) tensor([[1., 0.],
        [1., 0.],
        [0., 1.]])


## Definir la clase NET 


Se genera la clase NET para construir la red neuronal. Urilizamos 2 capas ocultas para dar mayor adaptabilidad al patron de los datos. Los nodos se establecieron como una reduccion uniforme desde la cantidad de entrada hasta la cantidad de salida. Como funcion de activacion tomamos la tangente hipebolica con el fin de tener una convergencia mas suave alejandonos de casos de sobre entrenamiento.

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20,14)
        self.fc2 = nn.Linear(14,8)
        self.fc3 = nn.Linear(8,2)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return x

Para el aprendizaje y evaluacion del modelo se utilizaran las funciones de decenso del gradiente y validacion cruzada en entropia. Se encontro que la validacion cruzada en entropia es la mejor funcion para realizar un modelo de clasificación, ya que toma entre las entradas de los resultados, la que es mayor y a esta la clasifica como 1 y a las otras como 0, hablando de las etiquetas en forma Dummies. Por parte del decenso del gradiente se encontro que tiene una convergencia mas suave y mejor adaptable.

In [14]:
model=Net()

optimizer= torch.optim.SGD(model.parameters(), lr=0.1)
criterion= nn.CrossEntropyLoss()

**Entrenar el modelo e ir imprimiendo los errores del train y la validación.**

In [15]:
def train_model(model,optimizer,loss_module,train_loader,valid_loader,num_epochs):
  
  valid_loss_min =np.inf  #Vamos a encontrar el menor valor de error de validación. Por eso la inicializmaos como 'infinito'
  
  for i in range(num_epochs):
    model.train()  #ponemos el modelo en modo entrenamiento. Es importante en otras arquitecturas como redes convolucionales.
    train_loss = 0.0
    v_loss = 0.0

    for data, target in train_loader:
         # Reiniciar los gradientes
        optimizer.zero_grad()

        # forward pass: calcular la salida para los datos de entrada..
        preds = model(data)
        #preds = preds.squeeze(dim=1)

        # calculate the batch loss
        loss = loss_module(preds, target)
        
        # backpropagation: cálculo de gradientes
        loss.backward()
   
        # actualizar los parámetros
        optimizer.step()

        # actualizar la cuenta de costos a lo largo de los lotes
        train_loss += loss.item()*data.size(0)
    # for data,labels in testloader:

    train_loss = train_loss/len(train_loader.dataset) 

    model.eval() #Ponemos el modelo en modo evaluación.
    
    for data,target in valid_loader:
      output=model(data)
      valid_loss= criterion(output, target)
      valid_loss += loss.item()*data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    #imprimir estadísticas de entrenamiento y validación
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        i, train_loss, valid_loss))
    

    #Guardamos el modelo con el menor error de validación.
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_bikeshare.pt')
        valid_loss_min = valid_loss
    

## Entrenamiento del modelo 

In [16]:
train_model(model, optimizer,criterion,train_data,val_data,50)

Epoch: 0 	Training Loss: 0.477658 	Validation Loss: 0.002274
Validation loss decreased (inf --> 0.002274).  Saving model ...
Epoch: 1 	Training Loss: 0.475175 	Validation Loss: 0.002331
Epoch: 2 	Training Loss: 0.475034 	Validation Loss: 0.002294
Epoch: 3 	Training Loss: 0.474122 	Validation Loss: 0.002277
Epoch: 4 	Training Loss: 0.474316 	Validation Loss: 0.002259
Validation loss decreased (0.002274 --> 0.002259).  Saving model ...
Epoch: 5 	Training Loss: 0.474283 	Validation Loss: 0.002248
Validation loss decreased (0.002259 --> 0.002248).  Saving model ...
Epoch: 6 	Training Loss: 0.474909 	Validation Loss: 0.002251
Epoch: 7 	Training Loss: 0.474978 	Validation Loss: 0.002247
Validation loss decreased (0.002248 --> 0.002247).  Saving model ...
Epoch: 8 	Training Loss: 0.475220 	Validation Loss: 0.002244
Validation loss decreased (0.002247 --> 0.002244).  Saving model ...
Epoch: 9 	Training Loss: 0.475539 	Validation Loss: 0.002243
Validation loss decreased (0.002244 --> 0.002243).

**Visualizamos los parametros del mejor modelo**

In [17]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[ 1.3514e-01, -2.6385e-02, -1.8962e-01,  1.4930e-01,  3.7810e-03,
          2.0476e-01,  1.0111e-01, -1.9228e-01,  5.7838e-02, -2.8177e-02,
         -5.9391e-02,  1.6011e-01,  1.8771e-01,  1.8701e-01,  1.9688e-01,
          1.1719e-01, -1.1609e-01, -7.2434e-02,  3.4893e-01,  7.6411e-02],
        [-1.2226e-01, -1.3628e-01,  1.4062e-01,  2.2223e-01,  7.4105e-02,
         -1.4121e-01, -9.3809e-02,  4.6838e-01, -1.3463e-01, -1.3405e-01,
         -4.8723e-02,  8.7199e-02, -3.2568e-02,  7.0329e-02,  8.9977e-02,
          3.4566e-01, -1.1178e-02, -2.4744e-01, -3.5712e-01,  1.5952e-01],
        [ 3.0448e-02,  2.1198e-01,  2.0869e-01, -4.6161e-02,  3.4241e-02,
          2.2690e-01, -8.4570e-02, -6.7171e-02, -2.9196e-02,  1.6485e-01,
         -9.9608e-02, -1.5857e-01, -1.9046e-01, -7.5946e-02,  7.9869e-02,
          6.8404e-02, -5.8451e-02,  2.8151e-01,  1.9200e-01,  2.1102e-01],
        [ 2.4891e-02,  4.8917e-02,  1.8544e-01,  1.6309e-02, -1.9673e-02,
          1.0005e-01,  7

### Evaluación del modelo utilizando el Test
La metrica que utilizamos para la evaluación del test fue el acurracy, lo elaboramos a mano e implementamos una nueva parte con el segundo for en donde replicamos la funcionalidad del croos entropy para tomar el output que tenga la mayor magnitud como 1 y el que tiene la menor ponerle 0. 

In [18]:
model.eval()
total, correct =0,0
for i, data in enumerate(test_data, 0):
    inputs, labels = data[0], data[1]
    
    # Poner cada imagen dentro del input
    inputs = inputs.view(-1, 20)
    
    # Pasar las predicciones
    outputs = model(inputs)
    # tomar la predicción con mayor valor y poner 1 como clasificación y a las otras 0
    for i in range(outputs.size(0)):
        maxi = torch.max(outputs[i])
        outputs[i] = outputs[i]==maxi
    # Realizamos manualmente el acurrancy.
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
    
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 94 %


El modelo demuestra ser bastante acertado al tener un 94% de datos correctos.

### Escoger una muestra aleatoria del DataSet
Vamos a tomar una muestra aleatoria con la función Sample y utilizar el modelo anterior para poder predecir el semintimiento de una entrada escogida aleatoriamente del DataSet inicial.

In [25]:
df_all = pd.concat([X,y],axis = 1)

sample = df_all.sample(1)
sample.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,happy,sad
177,0.181808,0.06061,0.177591,0.142939,0.232044,0.089105,1.363956,5.421496,0.938344,0.499332,...,0.129764,0.020752,0.262295,0.539062,0.0,2.734375,2.734375,0.155325,True,False


In [26]:
sample_y = sample[['happy','sad']]
sample_x = sample.drop(columns = ['happy','sad'])

In [27]:
#Pasamos nuestros datos sin la muestra para ser preparados.
sample_sec=MyDataset(sample_x, sample_y)

# Definimos el Sample Data
sample_data=DataLoader(
    sample_sec,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

criterion= nn.CrossEntropyLoss()
# Pasamos el Sample al modelo
for data, target in sample_data:
    output=model(data)
    for i in range(output.size(0)):
        maxi = torch.max(output[i])
        output[i] = output[i]==maxi
# Luego obtenemos la clasificación dada por nuestro modelo y la clasificación que se tenian en los datos.
print((output))
print((sample[['happy','sad']]))

tensor([[1., 0.]], grad_fn=<CopySlices>)
     happy    sad
177   True  False


## Conclusiones

* Obtuvimos una precisión del modelo del 94%, lo cual nos indica que es un buen modelo de clasificación utilizando una red neuronal con 2 capas lineales, capa de activación hiperbólica y entrenamiento basado en descenso del gradiente y validación cruzada en entropía.